<a href="https://colab.research.google.com/github/thiago-leonel-lab/notebooks-google/blob/main/notebook_auditoria_52978.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho de Auditoria 52978 - Assistida com IA
Este notebook realiza auditoria de fichas AVL usando IA generativa via API e bancos Supabase.
Cada célula é modular e comentada para melhor organização e colaboração.

In [ ]:
# Instalação de pacotes (executar no terminal se necessário)
!pip install -r requirements.txt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Carregamento de variáveis de ambiente
from config import conn_dicoi, conn_audit

## Extração de dados da base DICOI
Seleciona os 7 campos relevantes para auditoria.

In [ ]:
import pandas as pd
query = '''
SELECT cd_ficha, cd_cicl, cd_prf_gestor, tx_prf_gestor, pacr_conclusivo, analise, resultado
FROM avl_vw_ficha
JOIN avl_ficha_teste USING(cd_ficha)
LIMIT 400;
'''
df = pd.read_sql_query(query, conn_dicoi)
df.head()

## Divisão em lotes de 10 fichas
Evita sobrecarga na API e permite controle de processamento.

In [ ]:
def dividir_em_lotes(df, tamanho_lote=10):
    return [df[i:i + tamanho_lote] for i in range(0, len(df), tamanho_lote)]

lotes = dividir_em_lotes(df)

## Função para formatar fichas em texto plano
Prepara os dados para envio à API.

In [ ]:
def format_fichas_avl(df_lote):
    fichas_formatadas = []
    for _, row in df_lote.iterrows():
        ficha_texto = (
            f"Ficha AVL #{row['cd_ficha']}\n"
            f"cd_ficha: {row['cd_ficha']}\n"
            f"cd_cicl: {row['cd_cicl']}\n"
            f"cd_prf_gestor: {row['cd_prf_gestor']}\n"
            f"tx_prf_gestor: {row['tx_prf_gestor']}\n"
            f"analise: {row['analise']}\n"
            f"resultado: {row['resultado']}\n"
            f"pacr_conclusivo: {row['pacr_conclusivo']}"
        )
        fichas_formatadas.append(ficha_texto)
    return fichas_formatadas